In [12]:
# conda activate anndata

import re
import os
import sys
import numpy as np
import pandas as pd
import anndata as ad
from scipy.io import mmwrite
from functools import reduce
from scipy.sparse import csr_matrix

sys.path.append("/mnt/lareaulab/reliscu/code")

from junction2psi import *

In [13]:
os.chdir("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/bulk/hahn_2023/cortex")

## Metadata

In [14]:
sra_meta = pd.read_csv("../SraRunTable.csv")
sra_meta = sra_meta.loc[sra_meta['tissue'].str.contains("cortex")]

sample_meta = pd.read_csv("../BulkSeq_Aging_metadata.txt", sep="\t")
sample_meta = sample_meta.loc[sample_meta['tissueLong'].str.contains("cortex")]

In [15]:
sample_meta

,sampleID,tissueLong,organism,sex,age,strain,mouseID
115,BulkSeq_cor_1,Motor cortex,Mus Musculus,Female,12,C57BL/6JN,1
116,BulkSeq_cor_10,Motor cortex,Mus Musculus,Female,12,C57BL/6JN,10
117,BulkSeq_cor_11,Motor cortex,Mus Musculus,Male,26,C57BL/6JN,11
118,BulkSeq_cor_12,Motor cortex,Mus Musculus,Male,12,C57BL/6JN,12
119,BulkSeq_cor_13,Motor cortex,Mus Musculus,Female,12,C57BL/6JN,13
...,...,...,...,...,...,...,...
278,BulkSeq_ent_58,Entorhinal cortex,Mus Musculus,Male,28,C57BL/6JN,58
279,BulkSeq_ent_59,Entorhinal cortex,Mus Musculus,Male,28,C57BL/6JN,59
280,BulkSeq_ent_7,Entorhinal cortex,Mus Musculus,Female,12,C57BL/6JN,7
281,BulkSeq_ent_8,Entorhinal cortex,Mus Musculus,Male,26,C57BL/6JN,8


In [22]:
sra_meta['SampleID'] = "BulkSeq_cor_" + sra_meta["mouse_id_number"].astype("string")

In [25]:
sra_meta.loc[sra_meta['tissue'].str.contains("Ent"), 'SampleID'] = sra_meta.loc[sra_meta['tissue'].str.contains("Ent"), 'SampleID'].str.replace("cor", "ent")

In [8]:
sra_meta = sra_meta.groupby("mouse_id_number")[["Run"]].first()

In [28]:
sample_info = sra_meta.merge(sample_meta, left_on="SampleID", right_on="sampleID", how="left")

In [29]:
sample_info

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,strain_x,tissue,SampleID,sampleID,tissueLong,organism,sex_y,age,strain_y,mouseID
0,SRR21355217,RNA-Seq,300,1889509800,PRJNA875066,SAMN30602148,730275267,"TONY WYSS-CORAY, NEUROLOGY & NEUROLOGICAL SCIE...",public,"fastq,run.zq,sra",...,C57BL/6JN,Entorhinal cortex,BulkSeq_ent_17,BulkSeq_ent_17,Entorhinal cortex,Mus Musculus,Male,3,C57BL/6JN,17
1,SRR21355218,RNA-Seq,300,10976929800,PRJNA875066,SAMN30602149,4168379978,"TONY WYSS-CORAY, NEUROLOGY & NEUROLOGICAL SCIE...",public,"fastq,run.zq,sra",...,C57BL/6JN,Entorhinal cortex,BulkSeq_ent_16,BulkSeq_ent_16,Entorhinal cortex,Mus Musculus,Male,21,C57BL/6JN,16
2,SRR21355219,RNA-Seq,300,1387805100,PRJNA875066,SAMN30602150,533390758,"TONY WYSS-CORAY, NEUROLOGY & NEUROLOGICAL SCIE...",public,"fastq,run.zq,sra",...,C57BL/6JN,Entorhinal cortex,BulkSeq_ent_15,BulkSeq_ent_15,Entorhinal cortex,Mus Musculus,Male,12,C57BL/6JN,15
3,SRR21355224,RNA-Seq,300,6230791500,PRJNA875066,SAMN30602155,2320555077,"TONY WYSS-CORAY, NEUROLOGY & NEUROLOGICAL SCIE...",public,"fastq,run.zq,sra",...,C57BL/6JN,Entorhinal cortex,BulkSeq_ent_1,BulkSeq_ent_1,Entorhinal cortex,Mus Musculus,Female,12,C57BL/6JN,1
4,SRR21355241,RNA-Seq,300,6477564300,PRJNA875066,SAMN30602244,2380886548,"TONY WYSS-CORAY, NEUROLOGY & NEUROLOGICAL SCIE...",public,"fastq,run.zq,sra",...,C57BL/6JN,Motor cortex,BulkSeq_cor_35,BulkSeq_cor_35,Motor cortex,Mus Musculus,Male,18,C57BL/6JN,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,SRR21355379,RNA-Seq,300,6558137100,PRJNA875066,SAMN30602239,2485152926,"TONY WYSS-CORAY, NEUROLOGY & NEUROLOGICAL SCIE...",public,"fastq,run.zq,sra",...,C57BL/6JN,Motor cortex,BulkSeq_cor_4,BulkSeq_cor_4,Motor cortex,Mus Musculus,Female,12,C57BL/6JN,4
106,SRR21355380,RNA-Seq,300,1941993600,PRJNA875066,SAMN30602240,722636693,"TONY WYSS-CORAY, NEUROLOGY & NEUROLOGICAL SCIE...",public,"fastq,run.zq,sra",...,C57BL/6JN,Motor cortex,BulkSeq_cor_39,BulkSeq_cor_39,Motor cortex,Mus Musculus,Female,18,C57BL/6JN,39
107,SRR21355381,RNA-Seq,300,4429329300,PRJNA875066,SAMN30602241,1679518357,"TONY WYSS-CORAY, NEUROLOGY & NEUROLOGICAL SCIE...",public,"fastq,run.zq,sra",...,C57BL/6JN,Motor cortex,BulkSeq_cor_38,BulkSeq_cor_38,Motor cortex,Mus Musculus,Female,15,C57BL/6JN,38
108,SRR21355382,RNA-Seq,300,6441557400,PRJNA875066,SAMN30602242,2416645938,"TONY WYSS-CORAY, NEUROLOGY & NEUROLOGICAL SCIE...",public,"fastq,run.zq,sra",...,C57BL/6JN,Motor cortex,BulkSeq_cor_37,BulkSeq_cor_37,Motor cortex,Mus Musculus,Female,3,C57BL/6JN,37


In [31]:
sample_info.to_csv("hahn_2023_cortex_sampleinfo.csv", index=False)

## Gene counts: featureCounts

In [129]:
# samples = os.listdir("processed/featureCounts")
# samples = [s for s in samples if re.search("SR*", s)]

In [130]:
# cts_list = []
# tpm_list = []

# for ea in samples:
#     df = pd.read_csv(
#         f"processed/featureCounts/{ea}/{ea}.txt", 
#         header=1, sep="\t"
#     )
#     counts = df.iloc[:, -1]
#     rate = counts / df['Length'] * 1000
#     tpm = rate / rate.sum() * 1e6
    
#     cts_list.append(counts)
#     tpm_list.append(tpm)

In [131]:
# expr_tpm = pd.concat(tpm_list, axis=1)
# expr_counts = pd.concat(cts_list, axis=1)
# expr_tpm.columns = samples
# expr_counts.columns = samples
# expr_tpm.index = df['gene_name']
# expr_counts.index = df['gene_name']

In [132]:
# expr_counts.to_csv(f"{outdir}/hahn_2023_motor_cortex_STAR_featureCounts_gene_counts.csv")
# expr_tpm.to_csv(f"{outdir}/hahn_2023_motor_cortex_STAR_featureCounts_gene_TPM.csv")

## Gene counts: STAR

In [6]:
samples = os.listdir("processed/STAR")
samples = [s for s in samples if re.search("SR*", s)]

In [8]:
gene_length = pd.read_csv(
    "/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/bulk/gene_length.txt",
    sep="\t", header=None, names=["Gene", "Ensembl", "Type", "Length"]
)

In [9]:
cts_list = []
tpm_list = []

for ea in samples:
    star_output = pd.read_csv(
        f"processed/STAR/{ea}/ReadsPerGene.out.tab", 
        header=3, sep="\t", 
        names=["Ensembl", "unstranded", "first_strand", "second_strand"]
    )
    
    df = star_output.merge(
        gene_length, how="left",
        left_on="Ensembl", right_on="Ensembl", 
    )
    df = df.set_index("Gene", drop=True)
    
    counts = df['unstranded']
    rate = counts / df['Length'] * 1000
    tpm = rate / rate.sum() * 1e6
    
    cts_list.append(counts)
    tpm_list.append(tpm)

In [10]:
expr_tpm = pd.concat(tpm_list, axis=1)
expr_counts = pd.concat(cts_list, axis=1)

In [11]:
expr_tpm.columns = samples
expr_counts.columns = samples

In [12]:
expr_counts.to_csv("hahn_2023_cortex_STAR_gene_counts.csv")
expr_tpm.to_csv("hahn_2023_cortex_STAR_gene_TPM.csv")

## SJ counts

In [13]:
def process_SJ_line(line, idx, sj_counts):
    try:
        line = line.decode().rstrip().split('\t')
    except:
        line = line.rstrip().split('\t')

    if line[3] == '1':
        strand = '+'
    elif line[3] == '2':
        strand = '-'
    else:
        strand = 'REMOVE'

    intron_idx = line[0] + ':' + line[1] + '-' + line[2] + ':' + strand
    idx.append(intron_idx)
    sj_counts.append(int(line[6]))
    
    return idx, sj_counts

In [14]:
dtype = np.float64
intron_file = "/mnt/lareaulab/reliscu/data/GENCODE/GRCm39/psix_annotation/intron_file.tab.gz"
intron_table = read_intron_file(intron_file)

In [15]:
samples = os.listdir("processed/STAR")
samples = [s for s in samples if re.search("SR*", s)]

sj_list = []

colnames = ["chr", "intron_first_base", "intron_last_base", "strand", 
            "intron_motif", "junction_type", "n_unique_jxn_reads",
            "n_multi_jxn_reads", "max_align_overhang"]

for ea in samples:
    sj_name = []
    sj_counts = []
    with open(f"processed/STAR/{ea}/SJ.out.tab", "rb") as fh:
        for line in fh:
            sj_name, sj_counts = process_SJ_line(line, sj_name, sj_counts)
    sj_df = pd.DataFrame(sj_counts, index=sj_name, columns=[ea])
    sj_list.append(sj_df)

In [16]:
expr_sj = reduce(
    lambda x, y: x.merge(y, how="outer", left_index=True, right_index=True), 
    sj_list
)
expr_sj.shape

(1655997, 110)

In [17]:
intron_table['intron'] = intron_table['intron'].astype(str)
sj_table = pd.merge(intron_table, expr_sj, # Subset to SJs in annotated intron table
                    left_on="intron", how="left", 
                    right_index=True).fillna(0)
sj_table.shape

(157406, 111)

In [18]:
sj_table = sj_table.drop(columns=["intron"])

In [19]:
sj_table.head()

,SRR21355374,SRR21355671,SRR21355241,SRR21355346,SRR21355701,SRR21355367,SRR21355383,SRR21355217,SRR21355260,SRR21355668,...,SRR21355357,SRR21355250,SRR21355660,SRR21355376,SRR21355673,SRR21355243,SRR21355344,SRR21355249,SRR21355703,SRR21355697
ENSMUSG00000025900_ProteinCoding_1_I1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSMUSG00000025900_ProteinCoding_1_I2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ENSMUSG00000025900_ProteinCoding_1_SE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSMUSG00000025902_ProteinCoding_1_I1,5.0,0.0,4.0,2.0,10.0,14.0,6.0,0.0,3.0,4.0,...,10.0,1.0,8.0,13.0,0.0,3.0,3.0,7.0,5.0,7.0
ENSMUSG00000025902_ProteinCoding_1_I2,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [20]:
adata = ad.AnnData(
    X=sj_table.to_numpy().T,
    var=pd.DataFrame(index=sj_table.index),
    obs=pd.DataFrame(index=sj_table.columns),
)

In [21]:
adata.shape

(110, 157406)

In [22]:
adata.write_h5ad("hahn_2023_cortex_STAR_SJ_counts.h5ad")